In [54]:
import time
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [55]:
def get_soup(url:str, params:dict=dict()):
    r = requests.get(url, params)
    r.encoding = r.apparent_encoding
    if r.status_code != 200:
        raise Exception()
    return BeautifulSoup(r.text, "lxml")

def get_page_links(rubric_url):
    soup = get_soup(rubric_url)
    return [{"rubric": rubric_url.split("/")[-2], "href": j["href"], "title": j.text}
            for j in [i.find("a") for i in soup.find_all("h3")]
            if "article" in j["href"]]

def get_all_article_links():
    main_page_url = "https://www.buhgalteria.ru/"
    soup = get_soup(main_page_url)
    categories = [i["href"] for i in soup.find("ul", {"class": "dropnav-menu"}).find_all("a")]
    categories = [href for href in categories if "rubric" in href]

    all_page_links = list()
    all_page_hrefs = set()
    for rubric in categories:
        page_links = get_page_links(rubric_url=rubric)
        for link in page_links:
            link["href"] = link["href"].split("?")[0]
            if "https://www.buhgalteria.ru" not in link["href"]:
                link["href"] = "https://www.buhgalteria.ru" + link["href"]
            if link["href"] not in all_page_hrefs:
                all_page_hrefs.add(link["href"])
                all_page_links.append(link)
    return all_page_links


def main():
    URL = 'https://www.buhgalteria.ru'
    all_pages = get_all_article_links()
    news_data = {}

    soup = get_soup(URL)

    # Для каждой ссылки получаем информацию и записываем в news_data
    for page in all_pages:

        #link = news_links[i].get('href').split('?')[0]
        link = page["href"]
        name = link
        news_data[name] = {"title": page["title"], "category": page["rubric"]}
        soup = get_soup(link)

        # Переходим на страницу для дальнейшенго парсинга
        article = soup.find('div', class_='article')

        image = article.find('div', class_='article__main-image')

        article_text = " ".join(art.text for art in article.find_all('p'))

        # Заполняем полученными данными news_data
        news_data[name]['link'] = link
        #news_data[name]['date'] = date
        news_data[name]['text'] = " ".join(article_text.split())

    pd.DataFrame(news_data).transpose().reset_index(drop=True).to_csv("parced_data/buh_data.csv")



if __name__ == "__main__":
    main()

In [53]:
pd.read_csv("buh_data.csv", index_col=0)

,title,category,link,text
0,Как определять базу по НДС при аренде госимуще...,uchetiottchetnost,https://www.buhgalteria.ru/article/kak-opredel...,Если вы арендуете государственное или муниципа...
1,Коэффициенты дефляторы на 2023 год,uchetiottchetnost,https://www.buhgalteria.ru/article/koeffitsien...,Минэкономразвития обнародовало коэффициенты-де...
2,Как ограничить доступ к своим отчетам в ГИРБО ...,uchetiottchetnost,https://www.buhgalteria.ru/article/kak-ogranic...,Доступ к финансовой и учредительной информации...
3,"Минфин напомнил, как подтвердить расходы перви...",uchetiottchetnost,https://www.buhgalteria.ru/article/minfin-napo...,"Все расходы, принимаемые в уменьшение налогооб..."
4,Как определять остаточную стоимость ОС при аренде,uchetiottchetnost,https://www.buhgalteria.ru/article/kak-opredel...,С 2022 года обязателен к применению ФСБУ 25/20...
...,...,...,...,...
65,Реквизиты в чеках. Обязательные и дополнительные,sudebny_spory,https://www.buhgalteria.ru/article/rekvizity-v...,В одном из обзоров судебной практики ФНС включ...
66,Компенсация за плохой ремонт автомобиля ИП: мн...,sudebny_spory,https://www.buhgalteria.ru/article/kompensatsi...,Когда в автосервисе машину отремонтировали нек...
67,Признают ли мобилизацию форс-мажором?,organizatsiya_biznesa,https://www.buhgalteria.ru/article/priznayut-l...,Министерство финансов подготовило проект прави...
68,Новые ГОСТы для СИЗ с 1 октября 2022 года,organizatsiya_biznesa,https://www.buhgalteria.ru/article/novye-gosty...,С 1 октября 2022 года введены в действие 16 ст...


,title,category,link,text
0,Как определять базу по НДС при аренде госимуще...,uchetiottchetnost,https://www.buhgalteria.ru/article/kak-opredel...,Если вы арендуете государственное или муниципа...
1,Коэффициенты дефляторы на 2023 год,uchetiottchetnost,https://www.buhgalteria.ru/article/koeffitsien...,Минэкономразвития обнародовало коэффициенты-де...
2,Как ограничить доступ к своим отчетам в ГИРБО ...,uchetiottchetnost,https://www.buhgalteria.ru/article/kak-ogranic...,Доступ к финансовой и учредительной информации...
3,"Минфин напомнил, как подтвердить расходы перви...",uchetiottchetnost,https://www.buhgalteria.ru/article/minfin-napo...,"Все расходы, принимаемые в уменьшение налогооб..."
4,Как определять остаточную стоимость ОС при аренде,uchetiottchetnost,https://www.buhgalteria.ru/article/kak-opredel...,С 2022 года обязателен к применению ФСБУ 25/20...
...,...,...,...,...
65,Реквизиты в чеках. Обязательные и дополнительные,sudebny_spory,https://www.buhgalteria.ru/article/rekvizity-v...,В одном из обзоров судебной практики ФНС включ...
66,Компенсация за плохой ремонт автомобиля ИП: мн...,sudebny_spory,https://www.buhgalteria.ru/article/kompensatsi...,Когда в автосервисе машину отремонтировали нек...
67,Признают ли мобилизацию форс-мажором?,organizatsiya_biznesa,https://www.buhgalteria.ru/article/priznayut-l...,Министерство финансов подготовило проект прави...
68,Новые ГОСТы для СИЗ с 1 октября 2022 года,organizatsiya_biznesa,https://www.buhgalteria.ru/article/novye-gosty...,С 1 октября 2022 года введены в действие 16 ст...
